In [1]:
import cv2
import os
from math import ceil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ipynb.fs.defs.FeatureMatching as FeatureMatching
import ipynb.fs.defs.GraphBuilding as GraphBuilding
import ipynb.fs.defs.SimpleGraphStitching as SimpleGraphStitching
from IPython.display import display

In [ ]:
def compute_error(H_gt, H):
    return np.linalg.norm(np.eye(3)-H_gt@np.linalg.inv(H))

In [ ]:
def compute_avg_error(H_gt, H):
    return np.mean([compute_error(H_gt_node, H_node) for H_gt_node, H_node in zip(H_gt,H)])

In [ ]:
def compute_ground_truth(dataset_name,imgs,T_norm,matching_threshold,matches_th, idx_ref):
    matches_dict_gt, _ = FeatureMatching.get_feature_matches(dataset_name,
                        imgs,
                        T_norm,
                        matching_threshold = matching_threshold,
                        number_of_matches = 1,
                        matches_th = matches_th,
                        RANSACmaxIters = 2000,
                        save_output = False,
                        save_images = False,
                        noisy_matching = False,
                        verbose = False
                       )

    M_gt, _, _ = GraphBuilding.build_simple_graph_matrices(dataset_name,
                imgs,
                matches_dict_gt,
                verbose = False,
                save_output = False
               )

    H_gt, stitched_image_gt = SimpleGraphStitching.simple_graph_stitching(dataset_name,
                                imgs,
                                T_norm,
                                M_gt, 
                                idx_ref = idx_ref,
                                verbose = False,
                                save_output = False,
                                beautify = True,
                                warp_shape = [10000,10000])
    
    return H_gt, stitched_image_gt

In [ ]:
def plot_images(results, stitched_img_gt):
    
    imgs = [r["img"] for r in results]
    labels = [r["name"] for r in results]
    
    imgs.append(stitched_img_gt)
    labels.append("Ground truth")
    
    n = len(labels)
    
    columns = 2
    rows = ceil(n/columns)
    fig = plt.figure(figsize=(columns*20, rows*20))

    # ax enables access to manipulate each of subplots
    ax = []

    for i in range(n):
        # create subplot and append to ax
        ax.append( fig.add_subplot(rows, columns, i+1) )
        ax[-1].set_title(labels[i])  # set title
        plt.imshow(imgs[i])

    plt.show()  

In [1]:
def visualize_results(df, results, stitched_img_gt):
    
    display(df)
    
    plot_images(results, stitched_img_gt)

In [ ]:
def read_data(dataset_dir,
                df_file,
                dataset_name,
                imgs,
                T_norm,
                matching_threshold,
                matches_th, 
                idx_ref,
                results):
    index = ["number_of_matches", "noise_std"]
    gt_file = os.path.join(dataset_dir,"gt.npy")
    gt_img_file = os.path.join(dataset_dir,"gt_img.jpg")
    
    #Compute GT if not existing
    if os.path.isfile(gt_file):
        H_gt = np.load(gt_file)
        stitched_img_gt = cv2.cvtColor(cv2.imread(gt_img_file), cv2.COLOR_BGR2RGB)
        df = pd.read_csv(df_file, index_col = index)
    else:
        H_gt, stitched_img_gt = compute_ground_truth(dataset_name,imgs,T_norm,matching_threshold,matches_th, idx_ref)
        cv2.imwrite(gt_img_file, cv2.cvtColor(stitched_img_gt, cv2.COLOR_RGB2BGR))
        np.save(gt_file,H_gt)
        columns = index + [r["name"] for r in results] + ["Experiments"]
        df = pd.DataFrame(columns=columns)
        df.set_index(index,inplace=True)
        
    return H_gt, stitched_img_gt, df

In [ ]:
def run_benchmark(dataset_name,
                  imgs,
                  T_norm,
                  matching_threshold,
                  matches_th, 
                  idx_ref, 
                  results, 
                  number_of_matches, 
                  noise_std, 
                  benchmark_dir):
    
    dataset_dir = os.path.join(benchmark_dir,f"{dataset_name}_{idx_ref}_{matching_threshold}_{matches_th}")
  
    df_file = os.path.join(dataset_dir,"results.csv")
    os.makedirs(dataset_dir, exist_ok = True)
    
    H_gt, stitched_img_gt, df = read_data(dataset_dir,
                df_file,
                dataset_name,
                imgs,
                T_norm,
                matching_threshold,
                matches_th, 
                idx_ref,
                results)
    
    index = (number_of_matches, noise_std)
    if index in df.index:
        exp = df.loc[index,'Experiments'] + 1
        new_row = [df.loc[index,r["name"]] * ((exp-1)/exp) + compute_avg_error(H_gt,r["H"])*1/exp for r in results]
        new_row.append(exp)
    else:
        new_row = [compute_avg_error(H_gt,r["H"]) for r in results]
        new_row.append(1)
    
    df.loc[index,:] = new_row
    df.to_csv(df_file)
    
    visualize_results(df, results, stitched_img_gt)
